In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install pytorch-lightning
import pytorch_lightning as pl
print(pl.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
1.6.4


In [5]:
import pytorch_lightning as pl

pl.__version__

'1.6.4'

In [6]:
# Importing data
data = pd.read_csv('/content/drive/MyDrive/Tarea de Pytorch_Keras/4.Cancer_de_seno_Keras/data.csv')
del data['Unnamed: 32']

In [7]:
x = data.iloc[:,2:].values # extrae como arreglos numpy
y = data.iloc[:,1].values

In [28]:
from sklearn.preprocessing import *
from __future__ import absolute_import, division, print_function, unicode_literals
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)

In [78]:
x_train,

(array([[-0.52787029,  2.49821982, -0.59939466, ..., -1.74713139,
         -0.79044533, -0.91054389],
        [-0.55333608,  0.29431013, -0.60759343, ..., -0.62275667,
         -0.33646358, -0.83551633],
        [ 2.15452653,  0.40392257,  2.26525805, ...,  1.03846122,
         -0.11504791,  0.26488788],
        ...,
        [-1.3297598 , -0.21876938, -1.32088704, ..., -0.98271999,
         -0.718764  , -0.13637062],
        [-1.24940108, -0.24209117, -1.2835826 , ..., -1.74713139,
         -1.58690456, -1.01280367],
        [-0.74291476,  1.08958336, -0.71827692, ..., -0.2865488 ,
         -1.26354211,  0.19486216]]),)

In [35]:
x_train.shape

(512, 30)

In [30]:
x_test.shape

(57, 30)

In [32]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [33]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.profiler import Profiler, AdvancedProfiler
import torch.nn.functional as F

In [47]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch import nn, optim, rand, sum as tsum, reshape, save

In [70]:

from pytorch_lightning.callbacks import EarlyStopping
from torch import nn, optim, rand, sum as tsum, reshape, save
import torch.nn.functional as F
from pytorch_lightning import seed_everything, LightningModule, Trainer
from torch.utils.data import DataLoader, Dataset

In [85]:
class CustomDataset(Dataset):
    def __init__(self, samples=21000):
        self.dataset = rand(samples,30).cpu().float() 

    def __getitem__(self, index):
        return (self.dataset[index]).cpu().float()

    def __len__(self):
        return self.dataset.size()[0]

In [87]:
class OurModel(pl.LightningModule):
    def __init__(self):
        super(OurModel, self).__init__()
        # Network layers
        self.linear = nn.Linear(21000, 2048)
        self.linear2 = nn.Linear(2048, 1)
        self.output = nn.Sigmoid()
        # Hyper-parameters, that we will auto-tune using lightning!
        self.lr = 0.000001
        self.batch_size = 512

    def forward(self, x):
        x = self.linear(x)
        x = self.linear2(x)
        output = self.output(x)
        return  output 

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        loader = DataLoader(CustomDataset(samples=43210), batch_size=self.batch_size, shuffle=True)
        return loader

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.binary_cross_entropy(self(x), y)
        return {'loss': loss, 'log': {'train_loss': loss}}

    def val_dataloader(self):
        loader = DataLoader(CustomDataset(samples=1234), batch_size=self.batch_size, shuffle=False)
        return loader

    def validation_step(self, batch, batch_nb):
        x, y = batch
        loss = F.binary_cross_entropy(self(x), y)
        return {'val_loss': loss, 'log': {'val_loss': loss}}

    def validation_epoch_end(self, outputs):
        val_loss_mean = sum([o['val_loss'] for o in outputs]) / len(outputs)
        # show val_acc in progress bar but only log val_loss
        results = {'progress_bar': {'val_loss': val_loss_mean.item()}, 'log': {'val_loss': val_loss_mean.item()},
                   'val_loss': val_loss_mean.item()}
        print("OUR LR:",self.lr)
        return results

In [75]:
dm=CustomDataset()

In [88]:
model = OurModel()
dm=CustomDataset()
trainer = Trainer(max_epochs = 50)  
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/lightning_logs

  | Name    | Type    | Params
------------------------------------
0 | linear  | Linear  | 43.0 M
1 | linear2 | Linear  | 2.0 K 
2 | output  | Sigmoid | 0     
------------------------------------
43.0 M    Trainable params
0         Non-trainable params
43.0 M    Total params
172.048   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

ValueError: ignored